In [1]:
import pickle
from spread import *

EXP_RES_FILE = "data500network-200peers.pickle"
with open(EXP_RES_FILE, 'rb') as f:
    networks, results, scenarios = pickle.load(f)
print("loaded")

/home/yourmother/.local/lib/python3.9/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


done


In [13]:
def small_desc(s: SpreadScenario) -> str:
    if "random picker" in s.description:
        picker = "random"
    elif "reliability favor picker" in s.description:
        picker = "rel exp weight"
    elif "first picker" in s.description:
        picker = "rel first" 
        
    return f"({s.spread_n_peers}, {s.spread_tick_interval}, {picker})"
   

In [15]:
def ticks_to_time(ticks):
    secs = ticks * 0.5
    if secs < 60:
        return f"{secs}s"
    else:
        return f"{round(secs/60, 2)}min"

In [4]:
from dataclasses import dataclass
import pandas as pd


@dataclass
class AggregatedColumn:
    success_all: bool = True
    success_ratio: float = 0.0
    success_ratio_good_peers: float = 0.0
    
    repeated_msgs_per_tick: int = 0
    repeated_msgs_per_tick_good_peers: int = 0
    repeated_msgs_ratio: float = 0.0
    repeated_msgs_ratio_good_peers: float = 0.0
    
    avg_end_tick: float = 0
    avg_end_tick_good_peers: float = 0.0
    
    worst_case_end_tick: int = 0
    worst_case_end_tick_good_peers: int = 0
    

    
def set_no_val(col: AggregatedColumn):
    col.repeated_msgs_per_tick = -1
    col.repeated_msgs_ratio = -1
    col.repeated_msgs_ratio_good_peers = -1
    col.avg_end_tick = -1
    col.avg_end_tick_good_peers = -1
    
def df_of(key: float) -> pd.DataFrame:
    networks_results = results[key]
    
    columns = [AggregatedColumn() for _ in scenarios]
    
    for net_result in networks_results:
        for i, result in enumerate(net_result.scenario_results):
            if result.end_tick == 9999:
                columns[i].success_all = False
            
            if result.end_tick > columns[i].worst_case_end_tick:
                columns[i].worst_case_end_tick = result.end_tick
                
            if result.end_tick_good_peers > columns[i].worst_case_end_tick_good_peers:
                columns[i].worst_case_end_tick_good_peers = result.end_tick_good_peers
            
            if result.success_good_peers:
                columns[i].success_ratio_good_peers += 1
                
            if not result.success:
                continue
            columns[i].success_ratio += 1
            
            columns[i].repeated_msgs_per_tick += result.repeated_msgs / result.end_tick
            columns[i].repeated_msgs_per_tick_good_peers += result.repeated_msgs_good_peers / result.end_tick
            columns[i].repeated_msgs_ratio += result.repeated_msgs / (len(result.spread_edges) + result.repeated_msgs)
            
            columns[i].repeated_msgs_ratio_good_peers += result.repeated_msgs_good_peers / (result.repeated_msgs_good_peers + result.good_peers_notified)
            
            columns[i].avg_end_tick += result.end_tick
            columns[i].avg_end_tick_good_peers += result.end_tick_good_peers
            
            

    for col in columns:       
        if col.success_ratio: 
            col.repeated_msgs_per_tick /= col.success_ratio
            col.repeated_msgs_per_tick_good_peers /= col.success_ratio
            col.repeated_msgs_ratio /= col.success_ratio
            col.repeated_msgs_ratio_good_peers /= col.success_ratio                                         
            col.avg_end_tick /= col.success_ratio
            col.avg_end_tick_good_peers /= col.success_ratio
        else:
            set_no_val(col)
                                                                      
        col.success_ratio_good_peers /= len(networks_results)
        col.success_ratio /= len(networks_results)
        
    
    df = pd.DataFrame(columns) \
        .round(2)              \
        .rename({i : small_desc(s) for (i,s) in enumerate(scenarios)}) \
        
    return df

In [5]:
pd.set_option('display.max_rows', None)

df0 = df_of(0.0)
df0 = df0[df0["success_all"]]
df0 = df0.drop(columns=["success_all", "success_ratio", "success_ratio_good_peers", "repeated_msgs_per_tick_good_peers", "repeated_msgs_ratio_good_peers", "avg_end_tick_good_peers", "worst_case_end_tick_good_peers"])


df0["repeated_msgs_ratio"].describe()

count    200.00000
mean       0.74115
std        0.02793
min        0.68000
25%        0.71750
50%        0.75000
75%        0.77000
max        0.78000
Name: repeated_msgs_ratio, dtype: float64

In [18]:
x = df0.sort_values("repeated_msgs_per_tick", ascending=True).head(5)
x["avg_end_tick"] = x["avg_end_tick"].apply(ticks_to_time)
x["worst_case_end_tick"] = x["worst_case_end_tick"].apply(ticks_to_time)
x

,repeated_msgs_per_tick,repeated_msgs_ratio,avg_end_tick,worst_case_end_tick
"(1, 100, rel favor)",0.33,0.69,7.1min,45.86min
"(1, 100, random)",0.35,0.68,6.75min,29.21min
"(1, 100, rel first)",0.35,0.70,7.05min,45.08min
"(2, 250, rel first)",0.43,0.71,6.96min,41.73min
"(2, 250, rel favor)",0.44,0.70,6.68min,56.32min


In [20]:
x = df0.sort_values("repeated_msgs_per_tick", ascending=True).tail(5)
x["avg_end_tick"] = x["avg_end_tick"].apply(ticks_to_time)
x["worst_case_end_tick"] = x["worst_case_end_tick"].apply(ticks_to_time)
x

,repeated_msgs_per_tick,repeated_msgs_ratio,avg_end_tick,worst_case_end_tick
"(9, 1, random)",106.66,0.78,1.93s,6.5s
"(9, 1, rel first)",106.97,0.78,2.035s,11.0s
"(99999, 1, rel first)",112.72,0.78,1.93s,10.5s
"(99999, 1, rel favor)",112.84,0.78,1.89s,9.0s
"(99999, 1, random)",112.84,0.78,1.86s,9.5s


In [21]:
x = df0.sort_values("avg_end_tick", ascending=True).head(5)
x["avg_end_tick"] = x["avg_end_tick"].apply(ticks_to_time)
x["worst_case_end_tick"] = x["worst_case_end_tick"].apply(ticks_to_time)
x

,repeated_msgs_per_tick,repeated_msgs_ratio,avg_end_tick,worst_case_end_tick
"(99999, 1, random)",112.84,0.78,1.86s,9.5s
"(99999, 1, rel favor)",112.84,0.78,1.89s,9.0s
"(99999, 1, rel first)",112.72,0.78,1.93s,10.5s
"(9, 1, random)",106.66,0.78,1.93s,6.5s
"(9, 1, rel favor)",106.25,0.78,1.98s,5.5s


In [22]:
x = df0.sort_values("worst_case_end_tick", ascending=True).head(5)
x["avg_end_tick"] = x["avg_end_tick"].apply(ticks_to_time)
x["worst_case_end_tick"] = x["worst_case_end_tick"].apply(ticks_to_time)
x

,repeated_msgs_per_tick,repeated_msgs_ratio,avg_end_tick,worst_case_end_tick
"(9, 1, rel favor)",106.25,0.78,1.98s,5.5s
"(9, 1, random)",106.66,0.78,1.93s,6.5s
"(7, 1, random)",95.50,0.77,2.085s,7.5s
"(5, 1, rel favor)",79.90,0.77,2.445s,8.5s
"(99999, 1, rel favor)",112.84,0.78,1.89s,9.0s


In [ ]:
# Malicious environments analysis in cells below:

In [57]:
from dataclasses import dataclass
import pandas as pd


@dataclass
class AggregatedColumn:
    success_all: bool = True
    success_ratio: float = 0.0
    success_ratio_good_peers: float = 0.0
    
    repeated_msgs_per_tick: int = 0
    repeated_msgs_per_tick_good_peers: int = 0
    repeated_msgs_ratio: float = 0.0
    repeated_msgs_ratio_good_peers: float = 0.0
    
    avg_end_tick: float = 0
    avg_end_tick_good_peers: float = 0.0
    
    worst_case_end_tick: int = 0
    worst_case_end_tick_good_peers: int = 0
    

    
def set_no_val(col: AggregatedColumn):
    col.repeated_msgs_per_tick = -1
    col.repeated_msgs_ratio = -1
    col.repeated_msgs_ratio_good_peers = -1
    col.avg_end_tick = -1
    col.avg_end_tick_good_peers = -1
    
def df_of(key: float) -> pd.DataFrame:
    networks_results = results[key]
    
    columns = [AggregatedColumn() for _ in scenarios]
    
    for net_result in networks_results:
        for i, result in enumerate(net_result.scenario_results):
            if result.end_tick == 9999:
                columns[i].success_all = False
            
            if result.end_tick > columns[i].worst_case_end_tick:
                columns[i].worst_case_end_tick = result.end_tick
                
            if result.end_tick_good_peers > columns[i].worst_case_end_tick_good_peers:
                columns[i].worst_case_end_tick_good_peers = result.end_tick_good_peers
            
            if result.success_good_peers:
                columns[i].success_ratio_good_peers += 1
                
            if result.success:
                columns[i].success_ratio += 1
                
            # if not result.success:
            #     continue
            # columns[i].success_ratio += 1
            
            columns[i].repeated_msgs_per_tick += result.repeated_msgs / result.end_tick
            columns[i].repeated_msgs_per_tick_good_peers += result.repeated_msgs_good_peers / result.end_tick
            columns[i].repeated_msgs_ratio += result.repeated_msgs / (len(result.spread_edges) + result.repeated_msgs)
            
            if result.good_peers_notified != 0:
                columns[i].repeated_msgs_ratio_good_peers += result.repeated_msgs_good_peers / (result.repeated_msgs_good_peers + result.good_peers_notified)

            
            if result.success:
                columns[i].avg_end_tick += result.end_tick
            if result.success_good_peers:
                columns[i].avg_end_tick_good_peers += result.end_tick_good_peers
            
            

    for col in columns:
        col.avg_end_tick /= col.success_ratio
        col.avg_end_tick_good_peers /= col.success_ratio_good_peers
        
        col.repeated_msgs_per_tick /= len(networks_results)
        col.repeated_msgs_per_tick_good_peers /= len(networks_results)
        col.repeated_msgs_ratio /= len(networks_results)
        col.repeated_msgs_ratio_good_peers /= len(networks_results)                                         
                                                                      
        col.success_ratio_good_peers /= len(networks_results)
        col.success_ratio /= len(networks_results)
        
    df = pd.DataFrame(columns) \
        .round(2)              \
        .rename({i : small_desc(s) for (i,s) in enumerate(scenarios)}) \
        
    return df

In [68]:
df25 = df_of(0.25)
df25[df25["success_ratio_good_peers"] == 1.0]["repeated_msgs_ratio_good_peers"].describe()

count    201.000000
mean       0.726915
std        0.031023
min        0.660000
25%        0.700000
50%        0.730000
75%        0.750000
max        0.770000
Name: repeated_msgs_ratio_good_peers, dtype: float64

In [69]:
x = df25[df25["success_ratio_good_peers"] == 1.0].sort_values("repeated_msgs_per_tick_good_peers", ascending=True).head(5).drop(columns=["success_all", "success_ratio", "success_ratio_good_peers"])
x["avg_end_tick"] = x["avg_end_tick"].apply(ticks_to_time)
x["avg_end_tick_good_peers"] = x["avg_end_tick_good_peers"].apply(ticks_to_time)
x["worst_case_end_tick"] = x["worst_case_end_tick"].apply(ticks_to_time)
x["worst_case_end_tick_good_peers"] = x["worst_case_end_tick_good_peers"].apply(ticks_to_time)
x

,repeated_msgs_per_tick,repeated_msgs_per_tick_good_peers,repeated_msgs_ratio,repeated_msgs_ratio_good_peers,avg_end_tick,avg_end_tick_good_peers,worst_case_end_tick,worst_case_end_tick_good_peers
"(1, 100, random)",0.30,0.21,0.68,0.66,7.92min,7.65min,83.33min,49.23min
"(1, 100, rel favor)",0.30,0.21,0.69,0.68,7.86min,7.21min,83.33min,39.18min
"(1, 100, rel first)",0.33,0.25,0.70,0.70,7.48min,6.26min,83.33min,50.85min
"(2, 250, rel favor)",0.38,0.27,0.70,0.68,7.38min,6.62min,83.33min,56.27min
"(2, 250, random)",0.41,0.29,0.69,0.67,7.71min,7.33min,83.33min,77.1min


In [70]:
df50 = df_of(0.50)
df50[df50["success_ratio_good_peers"] == 0.99]["repeated_msgs_ratio_good_peers"].describe()

count    198.000000
mean       0.699545
std        0.034572
min        0.630000
25%        0.670000
50%        0.700000
75%        0.730000
max        0.760000
Name: repeated_msgs_ratio_good_peers, dtype: float64

In [71]:
x = df50[df50["success_ratio_good_peers"] == 0.99].sort_values("repeated_msgs_per_tick_good_peers", ascending=True).head(5).drop(columns=["success_all", "success_ratio", "success_ratio_good_peers"])
x["avg_end_tick"] = x["avg_end_tick"].apply(ticks_to_time)
x["avg_end_tick_good_peers"] = x["avg_end_tick_good_peers"].apply(ticks_to_time)
x["worst_case_end_tick"] = x["worst_case_end_tick"].apply(ticks_to_time)
x["worst_case_end_tick_good_peers"] = x["worst_case_end_tick_good_peers"].apply(ticks_to_time)
x

,repeated_msgs_per_tick,repeated_msgs_per_tick_good_peers,repeated_msgs_ratio,repeated_msgs_ratio_good_peers,avg_end_tick,avg_end_tick_good_peers,worst_case_end_tick,worst_case_end_tick_good_peers
"(1, 100, random)",0.23,0.10,0.68,0.63,11.1min,10.01min,83.33min,77.57min
"(1, 100, rel favor)",0.24,0.11,0.68,0.65,10.28min,7.98min,83.33min,80.07min
"(1, 100, rel first)",0.26,0.13,0.69,0.68,9.51min,6.3min,83.33min,56.75min
"(2, 250, rel favor)",0.30,0.13,0.69,0.65,9.85min,7.41min,83.33min,66.72min
"(2, 250, random)",0.30,0.13,0.69,0.64,11.08min,9.85min,83.33min,70.88min


In [72]:
df75 = df_of(0.75)
df75[df75["success_ratio_good_peers"] == 0.99]["repeated_msgs_ratio_good_peers"].describe()

count    167.000000
mean       0.646766
std        0.043246
min        0.560000
25%        0.610000
50%        0.650000
75%        0.680000
max        0.720000
Name: repeated_msgs_ratio_good_peers, dtype: float64

In [73]:
x = df75[df75["success_ratio_good_peers"] == 0.99].sort_values("repeated_msgs_per_tick_good_peers", ascending=True).head(5).drop(columns=["success_all", "success_ratio", "success_ratio_good_peers"])
x["avg_end_tick"] = x["avg_end_tick"].apply(ticks_to_time)
x["avg_end_tick_good_peers"] = x["avg_end_tick_good_peers"].apply(ticks_to_time)
x["worst_case_end_tick"] = x["worst_case_end_tick"].apply(ticks_to_time)
x["worst_case_end_tick_good_peers"] = x["worst_case_end_tick_good_peers"].apply(ticks_to_time)
x

,repeated_msgs_per_tick,repeated_msgs_per_tick_good_peers,repeated_msgs_ratio,repeated_msgs_ratio_good_peers,avg_end_tick,avg_end_tick_good_peers,worst_case_end_tick,worst_case_end_tick_good_peers
"(1, 50, rel favor)",0.36,0.08,0.68,0.60,9.01min,5.14min,83.33min,59.62min
"(2, 100, rel favor)",0.51,0.11,0.68,0.60,8.0min,4.12min,83.33min,77.53min
"(3, 250, rel favor)",0.46,0.11,0.69,0.62,11.37min,5.44min,83.33min,77.1min
"(2, 100, rel first)",0.49,0.13,0.69,0.64,8.12min,3.35min,83.33min,57.52min
"(1, 20, random)",0.85,0.17,0.68,0.56,5.1min,3.55min,83.33min,79.55min


In [76]:
# todo check the "fastest" strategies using "avg_end_tick_good_peers" and "worst_case_end_tick_good_peers" columns
x = df25[df25["success_ratio_good_peers"] == 1.0].sort_values("avg_end_tick_good_peers", ascending=True).head(5).drop(columns=["success_all", "success_ratio", "success_ratio_good_peers"])
x["avg_end_tick"] = x["avg_end_tick"].apply(ticks_to_time)
x["avg_end_tick_good_peers"] = x["avg_end_tick_good_peers"].apply(ticks_to_time)
x["worst_case_end_tick"] = x["worst_case_end_tick"].apply(ticks_to_time)
x["worst_case_end_tick_good_peers"] = x["worst_case_end_tick_good_peers"].apply(ticks_to_time)
x

,repeated_msgs_per_tick,repeated_msgs_per_tick_good_peers,repeated_msgs_ratio,repeated_msgs_ratio_good_peers,avg_end_tick,avg_end_tick_good_peers,worst_case_end_tick,worst_case_end_tick_good_peers
"(99999, 1, rel first)",102.74,79.02,0.77,0.77,2.025s,1.845s,83.33min,8.0s
"(99999, 1, rel favor)",103.81,76.19,0.77,0.76,1.95s,1.86s,83.33min,5.0s
"(9, 1, rel first)",97.77,75.16,0.77,0.77,2.105s,1.925s,83.33min,7.5s
"(99999, 1, random)",102.40,74.36,0.77,0.76,1.995s,1.95s,83.33min,13.0s
"(9, 1, rel favor)",96.58,70.77,0.77,0.76,2.075s,1.98s,83.33min,6.5s


In [41]:
df25[df25["success_ratio_good_peers"] == 1.0].sort_values("worst_case_end_tick_good_peers", ascending=True).head(5)

,success_all,success_ratio,success_ratio_good_peers,repeated_msgs_per_tick,repeated_msgs_per_tick_good_peers,repeated_msgs_ratio,repeated_msgs_ratio_good_peers,avg_end_tick,avg_end_tick_good_peers,worst_case_end_tick,worst_case_end_tick_good_peers
"(99999, 1, rel favor)",False,1.0,1.0,103.81,76.19,0.77,0.76,3,3,9999,10
"(9, 1, rel favor)",False,1.0,1.0,96.58,70.77,0.77,0.76,4,3,9999,13
"(7, 1, random)",False,1.0,1.0,86.20,62.47,0.76,0.75,4,4,9999,14
"(9, 1, random)",False,1.0,1.0,95.80,69.46,0.76,0.75,4,3,9999,15
"(9, 1, rel first)",False,1.0,1.0,97.77,75.16,0.77,0.77,4,3,9999,15


In [77]:
# todo check the "fastest" strategies using "avg_end_tick_good_peers" and "worst_case_end_tick_good_peers" columns
x = df50[df50["success_ratio_good_peers"] == 0.99].sort_values("avg_end_tick_good_peers", ascending=True).head(5).drop(columns=["success_all", "success_ratio", "success_ratio_good_peers"])
x["avg_end_tick"] = x["avg_end_tick"].apply(ticks_to_time)
x["avg_end_tick_good_peers"] = x["avg_end_tick_good_peers"].apply(ticks_to_time)
x["worst_case_end_tick"] = x["worst_case_end_tick"].apply(ticks_to_time)
x["worst_case_end_tick_good_peers"] = x["worst_case_end_tick_good_peers"].apply(ticks_to_time)
x

,repeated_msgs_per_tick,repeated_msgs_per_tick_good_peers,repeated_msgs_ratio,repeated_msgs_ratio_good_peers,avg_end_tick,avg_end_tick_good_peers,worst_case_end_tick,worst_case_end_tick_good_peers
"(9, 1, rel first)",79.74,41.06,0.76,0.76,2.635s,2.01s,83.33min,9.5s
"(99999, 1, rel first)",84.05,43.46,0.76,0.76,2.71s,2.02s,83.33min,28.0s
"(99999, 1, rel favor)",84.29,40.27,0.75,0.74,2.495s,2.055s,83.33min,33.0s
"(9, 2, rel first)",66.13,35.33,0.75,0.75,3.555s,2.195s,83.33min,15.5s
"(7, 1, rel first)",72.22,37.11,0.76,0.76,2.8s,2.205s,83.33min,23.0s


In [43]:
df50[df50["success_ratio_good_peers"] == 0.99].sort_values("worst_case_end_tick_good_peers", ascending=True).head(5)

,success_all,success_ratio,success_ratio_good_peers,repeated_msgs_per_tick,repeated_msgs_per_tick_good_peers,repeated_msgs_ratio,repeated_msgs_ratio_good_peers,avg_end_tick,avg_end_tick_good_peers,worst_case_end_tick,worst_case_end_tick_good_peers
"(7, 1, rel favor)",False,0.99,0.99,71.54,33.92,0.75,0.74,5,4,9999,18
"(9, 1, rel first)",False,0.99,0.99,79.74,41.06,0.76,0.76,5,4,9999,19
"(3, 1, rel favor)",False,0.99,0.99,42.75,19.74,0.74,0.72,8,6,9999,29
"(9, 2, rel first)",False,0.99,0.99,66.13,35.33,0.75,0.75,7,4,9999,31
"(5, 1, random)",False,0.99,0.99,59.12,26.76,0.75,0.71,6,5,9999,34


In [78]:
# todo check the "fastest" strategies using "avg_end_tick_good_peers" and "worst_case_end_tick_good_peers" columns
x = df75[df75["success_ratio_good_peers"] == 0.99].sort_values("avg_end_tick_good_peers", ascending=True).head(5).drop(columns=["success_all", "success_ratio", "success_ratio_good_peers"])
x["avg_end_tick"] = x["avg_end_tick"].apply(ticks_to_time)
x["avg_end_tick_good_peers"] = x["avg_end_tick_good_peers"].apply(ticks_to_time)
x["worst_case_end_tick"] = x["worst_case_end_tick"].apply(ticks_to_time)
x["worst_case_end_tick_good_peers"] = x["worst_case_end_tick_good_peers"].apply(ticks_to_time)
x

,repeated_msgs_per_tick,repeated_msgs_per_tick_good_peers,repeated_msgs_ratio,repeated_msgs_ratio_good_peers,avg_end_tick,avg_end_tick_good_peers,worst_case_end_tick,worst_case_end_tick_good_peers
"(99999, 1, rel first)",65.47,17.48,0.74,0.72,5.555s,2.17s,83.33min,44.0s
"(99999, 1, rel favor)",64.62,15.57,0.74,0.70,5.6s,2.27s,83.33min,24.5s
"(9, 1, rel first)",61.19,16.22,0.74,0.72,6.205s,2.285s,83.33min,36.0s
"(9, 1, rel favor)",61.33,14.67,0.74,0.70,5.205s,2.285s,83.33min,23.0s
"(7, 1, rel first)",55.97,14.72,0.74,0.72,5.825s,2.365s,83.33min,26.0s


In [45]:
# todo check the "fastest" strategies using "avg_end_tick_good_peers" and "worst_case_end_tick_good_peers" columns
df75[df75["success_ratio_good_peers"] == 0.99].sort_values("worst_case_end_tick_good_peers", ascending=True).head(5)

,success_all,success_ratio,success_ratio_good_peers,repeated_msgs_per_tick,repeated_msgs_per_tick_good_peers,repeated_msgs_ratio,repeated_msgs_ratio_good_peers,avg_end_tick,avg_end_tick_good_peers,worst_case_end_tick,worst_case_end_tick_good_peers
"(9, 1, rel favor)",False,0.96,0.99,61.33,14.67,0.74,0.70,10,4,9999,46
"(7, 1, rel favor)",False,0.96,0.99,55.65,13.18,0.74,0.70,11,4,9999,46
"(99999, 1, rel favor)",False,0.96,0.99,64.62,15.57,0.74,0.70,11,4,9999,49
"(9, 2, rel first)",False,0.96,0.99,47.75,13.29,0.73,0.71,16,5,9999,52
"(7, 1, rel first)",False,0.96,0.99,55.97,14.72,0.74,0.72,11,4,9999,52
